# HW5 - Transforming vote tallies
 
Given the election is still ongoing I thought I'd make this homework give some insight into the transformations that are underlying all the voting dashboards you've undoubtedly seen all over the news.  
 
The dataset for this week contains voting outcomes for different batches of votes in the battleground states.  The data consist of the following:
* The state of the votes
* The time the voting results were reported
* The number of votes in the batch (new_votes)
* The number of those new votes that were for Joe Biden (votes_biden)
 
The key metric of interest is in how each candidate, Biden and Trump, are doing over time in terms of the percentage of votes they're getting with each batch.  This is easier to think of compared to raw numbers for a variety of reasons.  One is that the number of votes varies state-to-state and batch-to-batch, so just thinking about raw numbers doesn't inform very much.  The other is that people are often thinking about the percentage a candidate needs to take the lead or win the state.  If each batch is hitting at or above that percentage, then they have a good chance of winning the state.  
 
The goal of this homework is to generate the percentages of votes in each state that are going to Trump and Biden **on and hourly reporting basis**. Each line is a batch of votes that were reported at a given time.  Sometimes there are multiple in an hour.  We want to report a coarser resolution which is why we want to group by hours and days.  
 
Given you have only the variables listed above, you're going to need to do a few things to make this happen:
* Import your data and apply a timestamp
* Make a column for the number of votes received by Trump
* Aggregate your data to get the number of votes for each on a daily and hourly basis
* Create columns of the percentages of votes each have received in the aggregated data
 
**NOTE 1** - This is by every measure 'small' data, but it needs to be coded in pyspark on databricks.
**NOTE 2** - These are all votes that were reported after the initial waves of in-person and early reported mail-in votes.  So if you explore you might see that the total votes in this dataset don't seem to match up with what you'll find elsewhere.  That's only because those early votes aren't added in.  These are just batch totals of later reported batches.

## Importing your data - 1 point
 
The URL to import your data is "https://ista322data.s3-us-west-1.amazonaws.com/votes_2020_hw5.txt"
 
You need to import the data with the appropriate options applied.  You will need to apply a timestampFormat option as well, but it's best to import the data first, look at the format, and then create your timestamp string.  Note, this is fractional seconds in the timestamp. <a href="https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html" target="_blank">Remember you can see a full list of datetime formatting here</a>
 
Also, although it's a text file you can import it as a CSV and specify a delimiter as an option.  
 
Call the imported dataframe 'votes'

In [0]:
# Make a filepath
import pyspark
from pyspark import SparkFiles

url = "https://ista322data.s3-us-west-1.amazonaws.com/votes_2020_hw5.txt"
spark.sparkContext.addFile(url)
fp = "file://"+SparkFiles.get("votes_2020_hw5.txt")

In [0]:
# import as votes
votes = (spark.read
         .option('header', True)
         .option('inferSchema', True)
         .option('delimiter', ',')
         .option("timestampFormat", "yyyy-MM-dd HH:mm:ss")
         .csv(fp))

In [0]:
# Check data
votes.display()

state,timestamp,new_votes,votes_biden
Alaska (EV: 3),2020-11-09 19:14:19.894,18939,7779.0
Alaska (EV: 3),2020-11-04 18:40:57.265,39556,11448.0
Alaska (EV: 3),2020-11-04 13:28:22.084,0,0.0
Arizona (EV: 11),2020-11-10 02:18:36.294,6397,3127.0
Arizona (EV: 11),2020-11-10 00:51:18.070,600,232.0
Arizona (EV: 11),2020-11-10 00:02:51.838,591,84.0
Arizona (EV: 11),2020-11-09 22:23:05.122,3437,1049.0
Arizona (EV: 11),2020-11-09 21:15:22.385,787,189.0
Arizona (EV: 11),2020-11-09 18:40:02.044,424,250.0
Arizona (EV: 11),2020-11-09 18:30:48.110,145,114.0


## Create number of votes for trump - 1 points
 
Just like it sounds.  Create a column in votes called 'votes_trump' that has the number of votes he received in the batch.
 
You'll need to import all the functions from pyspark.sql.functions to do this and later steps.

In [0]:
# make votes_trump
from pyspark.sql.functions import *

votes = votes.withColumn("votes_trump", votes['new_votes'] - votes['votes_biden'])

In [0]:
# Check
votes.display()

state,timestamp,new_votes,votes_biden,votes_trump
Alaska (EV: 3),2020-11-09 19:14:19.894,18939,7779.0,11160.0
Alaska (EV: 3),2020-11-04 18:40:57.265,39556,11448.0,28108.0
Alaska (EV: 3),2020-11-04 13:28:22.084,0,0.0,0.0
Arizona (EV: 11),2020-11-10 02:18:36.294,6397,3127.0,3270.0
Arizona (EV: 11),2020-11-10 00:51:18.070,600,232.0,368.0
Arizona (EV: 11),2020-11-10 00:02:51.838,591,84.0,507.0
Arizona (EV: 11),2020-11-09 22:23:05.122,3437,1049.0,2388.0
Arizona (EV: 11),2020-11-09 21:15:22.385,787,189.0,598.0
Arizona (EV: 11),2020-11-09 18:40:02.044,424,250.0,174.0
Arizona (EV: 11),2020-11-09 18:30:48.110,145,114.0,31.0


## Aggregate data - 2 points
 
Time to group your data. You're going to need to group by day, hour, and then state to get the totals.  I showed you how to group by a single column that's not a datetime in the previous lesson.  Do do more than one you just add them in separated by commas.  The only issue is that you need to extract day and hour from the timestamp.  To do this for the day of the month your grouping variable would be `dayofmonth('timestamp')`.  I'll let you figure out how to do it for hour.  The state should be easy.
 
For the aggregation you need to calculate the sums that will allow you to get the percentage of votes that went to Biden vs Trump in a given hour. 
 
Call the resulting dataframe votes_hourly

In [0]:
# make votes_hourly

votes_hourly = votes.groupBy(dayofmonth('timestamp'), hour('timestamp'), 'state').sum('new_votes', 'votes_biden', 'votes_trump')
votes_hourly = votes_hourly.withColumnRenamed("dayofmonth(timestamp)", "day") \
  .withColumnRenamed("hour(timestamp)", "hour")\
  .withColumnRenamed("dayofmonth(timestamp)", "day")\
  .withColumnRenamed("sum(new_votes)", "new_votes")\
  .withColumnRenamed("sum(votes_biden)", "votes_biden")\
  .withColumnRenamed("sum(votes_trump)", "votes_trump")
votes_hourly.display()

day,hour,state,new_votes,votes_biden,votes_trump
6,6,Pennsylvania (EV: 20),5878,5142.0,736.0
4,20,Pennsylvania (EV: 20),154565,126886.0,27679.0
5,15,Georgia (EV: 16),1203,824.0,379.0
6,0,Georgia (EV: 16),20187,13123.0,7064.0
6,1,Arizona (EV: 11),32786,15803.0,16983.0
5,23,Arizona (EV: 11),12227,4729.0,7498.0
4,14,Georgia (EV: 16),78,67.0,11.0
7,23,North Carolina (EV: 15),199,113.0,86.0
4,16,Pennsylvania (EV: 20),94133,68492.0,25641.0
8,23,Arizona (EV: 11),23601,10562.0,13039.0


## Calculating percentage of votes per hour - 2 points
 
Now go and make your columns of the percentage of votes received each hour by each candidate. Call these 'percent_biden' and 'percent_trump'

In [0]:
votes_hourly = votes_hourly.withColumn('percent_biden', votes_hourly['votes_biden'] / votes_hourly['new_votes'])
votes_hourly = votes_hourly.withColumn('percent_trump', votes_hourly['votes_trump'] / votes_hourly['new_votes'])
votes_hourly.display()

day,hour,state,new_votes,votes_biden,votes_trump,percent_biden,percent_trump
6,6,Pennsylvania (EV: 20),5878,5142.0,736.0,0.8747873426335488,0.12521265736645118
4,20,Pennsylvania (EV: 20),154565,126886.0,27679.0,0.8209232361789538,0.17907676382104618
5,15,Georgia (EV: 16),1203,824.0,379.0,0.684954280964256,0.31504571903574397
6,0,Georgia (EV: 16),20187,13123.0,7064.0,0.6500718284044187,0.3499281715955813
6,1,Arizona (EV: 11),32786,15803.0,16983.0,0.4820045141218813,0.5179954858781187
5,23,Arizona (EV: 11),12227,4729.0,7498.0,0.386766991085303,0.613233008914697
4,14,Georgia (EV: 16),78,67.0,11.0,0.8589743589743589,0.14102564102564102
7,23,North Carolina (EV: 15),199,113.0,86.0,0.5678391959798995,0.4321608040201005
4,16,Pennsylvania (EV: 20),94133,68492.0,25641.0,0.7276088088130623,0.2723911911869376
8,23,Arizona (EV: 11),23601,10562.0,13039.0,0.44752341002499896,0.552476589975001


## Rounding values - 1 point
 
Those percent_biden and percent_trump columns have too many digits.  You can import a rounding `round()` function from pyspark using `from pyspark.sql.functions import round`.  Go and import that function and then apply round to those columns.  It's fine to overwrite them with the rounded values.

In [0]:
from pyspark.sql.functions import round

votes_hourly = votes_hourly.withColumn('percent_biden', round(votes_hourly['percent_biden'], 2))
votes_hourly = votes_hourly.withColumn('percent_trump', round(votes_hourly['percent_trump'], 2))
votes_hourly.display()

day,hour,state,new_votes,votes_biden,votes_trump,percent_biden,percent_trump
6,6,Pennsylvania (EV: 20),5878,5142.0,736.0,0.87,0.13
4,20,Pennsylvania (EV: 20),154565,126886.0,27679.0,0.82,0.18
5,15,Georgia (EV: 16),1203,824.0,379.0,0.68,0.32
6,0,Georgia (EV: 16),20187,13123.0,7064.0,0.65,0.35
6,1,Arizona (EV: 11),32786,15803.0,16983.0,0.48,0.52
5,23,Arizona (EV: 11),12227,4729.0,7498.0,0.39,0.61
4,14,Georgia (EV: 16),78,67.0,11.0,0.86,0.14
7,23,North Carolina (EV: 15),199,113.0,86.0,0.57,0.43
4,16,Pennsylvania (EV: 20),94133,68492.0,25641.0,0.73,0.27
8,23,Arizona (EV: 11),23601,10562.0,13039.0,0.45,0.55


## SQL query - 3 points
 
Now let's leverage those SQL powers to summarize our data a bit more within arizona.  I want you to write two queries.  
1. The first should be just getting all data from Arizona.  FYI, the state name for Arizona is 'Arizona (EV: 11)'
2. The second should get the daily percentage of votes going to trump.  He needs to be averaging around 60% a day to catch up.  Is he?
 
Remember you need to register your dataframe as a table to access it!

In [0]:
# Uncomment and run if already existing
#spark.catalog.dropGlobalTempView("votes_hourly")

In [0]:
votes_hourly.createGlobalTempView('votes_hourly')

In [0]:
# query 1
sq = """SELECT * FROM global_temp.votes_hourly
        WHERE state = "Arizona (EV: 11)"
     """
spark.sql(sq).display()

day,hour,state,new_votes,votes_biden,votes_trump,percent_biden,percent_trump
6,1,Arizona (EV: 11),32786,15803.0,16983.0,0.48,0.52
5,23,Arizona (EV: 11),12227,4729.0,7498.0,0.39,0.61
8,23,Arizona (EV: 11),23601,10562.0,13039.0,0.45,0.55
6,19,Arizona (EV: 11),5043,1077.0,3966.0,0.21,0.79
6,20,Arizona (EV: 11),6666,2730.0,3936.0,0.41,0.59
7,0,Arizona (EV: 11),2976,647.0,2329.0,0.22,0.78
4,13,Arizona (EV: 11),0,0.0,0.0,null,null
5,0,Arizona (EV: 11),5614,2706.0,2908.0,0.48,0.52
8,17,Arizona (EV: 11),244,93.0,151.0,0.38,0.62
7,2,Arizona (EV: 11),79395,32383.0,47012.0,0.41,0.59


In [0]:
# query 2
sq = """SELECT day, ROUND(AVG(votes_trump / new_votes), 2) as daily_trump_percent FROM global_temp.votes_hourly
        GROUP BY day
        ORDER BY day
     """
spark.sql(sq).display()

day,daily_trump_percent
4,0.38
5,0.41
6,0.38
7,0.39
8,0.37
9,0.54
10,0.45


In [0]:
# No he is not as he getting around 40% daily on average